<a href="https://colab.research.google.com/github/julieannaqi/julieannaqi/blob/main/birch_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn

In [2]:
!pip install memory_profiler

In [3]:
import pandas as pd

df = pd.read_csv('/content/sampledata.csv')

# Display the contents of the CSV file
print(df)

                          ioc_match_id     event_time       src_host  \
0      1773_362000776412_0_60382090314  1701618689736  10.15.149.168   
1      1773_361424542980_0_59299513186  1701589169373  10.15.149.168   
2      1773_361424542980_0_59327455359  1701589169373  10.15.149.168   
3      1773_361424542980_0_59327639236  1701589169373  10.15.149.168   
4      1773_362000776412_0_60440942415  1701618689736  10.15.149.168   
...                                ...            ...            ...   
99995  1773_361569283028_0_60382090483  1701596405801  10.20.100.250   
99996  1773_361569283028_0_60724428162  1701596405801  10.20.100.250   
99997  1773_361569283121_0_60362960162  1701596405801   10.20.106.39   
99998  1773_361569283121_0_60374602795  1701596405801   10.20.106.39   
99999  1773_361569283121_0_60650052021  1701596405801   10.20.106.39   

            indicator  confidence severity indicator_type  originator  \
0        77.90.185.71          87      low             ip     

In [4]:
df.shape

(100000, 23)

In [5]:
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, silhouette_score
import h5py
import pyarrow as pa
from sklearn.cluster import OPTICS, cluster_optics_dbscan
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.utils.metaestimators import if_delegate_has_method
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.cluster import Birch

In [6]:
def calculate_metrics(data, predicted_labels):
        # homogeneity = homogeneity_score(true_labels, predicted_labels)
        # completeness = completeness_score(true_labels, predicted_labels)
        # v_measure = v_measure_score(true_labels, predicted_labels)
        # adj_rand_index = adjusted_rand_score(true_labels, predicted_labels)
        # adj_mutual_info = adjusted_mutual_info_score(true_labels, predicted_labels)
        # we don't know the true label at the moment, so we can only return the sillhouette score.
        silhouette_coeff = silhouette_score(data, predicted_labels)

        # return {
        #     "Homogeneity": homogeneity,
        #     "Completeness": completeness,
        #     "V-Measure": v_measure,
        #     "Adjusted Rand Index": adj_rand_index,
        #     "Adjusted Mutual Information": adj_mutual_info,
        #     "Silhouette Coefficient": silhouette_coeff
        # }
        return silhouette_coeff


In [7]:
df = df.sample(n=500, random_state=42)
non_numeric_cols = df.select_dtypes(exclude=['number']).columns.tolist()

# Numeric columns for imputation and scaling
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()

# ColumnTransformer to handle different transformations for different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), non_numeric_cols),
        ('imputer', SimpleImputer(strategy='constant', fill_value=-1), numeric_cols),  # Apply imputer to numeric columns
        ('scaler', StandardScaler(), numeric_cols)  # Apply scaler to numeric columns
    ]
)

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

# Fit and transform your data using the pipeline
processed_data = pipeline.fit_transform(df)

In [8]:
# birch works better with dense matrices
dense_matrix = processed_data.toarray()

In [15]:
%%time
%load_ext memory_profiler
%memit
from sklearn.cluster import Birch

birch = Birch(threshold=0.9, branching_factor=50, n_clusters=20, compute_labels=True)

labels = birch.fit_predict(dense_matrix)

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 262.75 MiB, increment: 0.00 MiB
CPU times: user 297 ms, sys: 138 ms, total: 435 ms
Wall time: 420 ms


In [16]:
print(f'the silhouette score for birch was {calculate_metrics(processed_data, labels)}')

the silhouette score for birch was 0.9635267822231072
